In [1]:
import jsonlines
import tqdm
from datetime import datetime
import nltk
nltk.download('stopwords')
import spacy
spacy.load('en')
from spacy.lang.en import English
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
parser = English()
en_stop = set(nltk.corpus.stopwords.words('english'))
from gensim import corpora
import pickle
import gensim

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rgopikrishna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
timestart = datetime.now()
data = {}

with open('aylien-covid-news.jsonl', 'rb') as f:
    for item in jsonlines.Reader(f):
        data[item['id']] = item['body']
        
duration = datetime.now() - timestart

In [3]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.is_stop:
            continue
        elif token.is_digit:
            continue
        elif token.is_punct:
            continue
        elif 'covid' in token.lower_ or 'virus' in token.lower_ or 'corona' in token.lower_:
            continue
        elif token.lower_ in en_stop:
            continue
        else:
            lda_tokens.append(ps.stem(token.lower_))
    return lda_tokens

In [8]:
tokenizeddocs = {}
for key in data:
    tokenizeddocs[key] = tokenize(data[key])

In [9]:
with open('bowformat_articles.pickle', 'wb') as filehandle:
    pickle.dump(tokenizeddocs, filehandle)

In [10]:
listtokenarticles = list(tokenizeddocs.values())

In [11]:
dictionary = corpora.Dictionary(listtokenarticles)

In [12]:
corpus = [dictionary.doc2bow(text) for text in listtokenarticles]

In [13]:
with open('corput_bow.pickle', 'wb') as filehandle:
    pickle.dump(corpus, filehandle)

In [14]:
dictionary.save('dictionary.gensim')

In [ ]:
import gensim